In [26]:
type Array a = Int -> a

In [27]:
emptyArray :: Array a
emptyArray i = error("null at " ++ show i)


getIndex :: Int -> Array a -> a
getIndex n ar = ar n


putIndex :: Int -> a -> Array a -> Array a
putIndex n x ar = \y -> if y == n then x else ar y


removeIndex :: Int -> Array a -> Array a
removeIndex n ar = \y -> if y == n then emptyArray n else ar y

-- getIndex 4 $ removeIndex 2 $ putIndex 2 "moin" $ putIndex 4 "hallo" emptyArray
-- (putIndex 4 "hi" $ putIndex 1 "moin" $ putIndex 2 "wasgeht" emptyArray) 4
-- (if index == 4 then hi else (if index == 1 then moin else (if index == 2 then wasgeht else error)))

let a = putIndex 1 "moin" $ putIndex 2 "wasgeht" emptyArray
let b = removeIndex 1 a
let c = removeIndex 2 a

a 1
-- (putIndex 1 "moin" $ putIndex 2 "wasgeht" emptyArray) 1
-- b 2
-- c 2

"moin"

In [28]:
newtype FList a = FList ([a] -> [a])


fromFList :: FList a -> [a]
fromFList (FList x) = x []

toFList :: [a] -> FList a
toFList xs = FList (xs ++)

emptyList :: FList a
emptyList = toFList []

In [29]:
instance Semigroup (FList a) where
    (<>) :: FList a -> FList a -> FList a
    (<>) (FList fl) (FList fr) = FList (fl . fr)
    --                                  \y -> fr (fl y)

instance Monoid (FList a) where
    mempty = FList $ const []
    -- mempty = FList $ \_ -> []
    mappend = (<>)

instance Show a => Show (FList a) where
    show :: FList a -> String
    show fl = show (fromFList fl)

-- :t (.)
-- (.) fr fl = \y -> fr (fl y)

In [30]:
singleton :: a -> FList a
singleton x = toFList [x]

cons :: a -> FList a -> FList a
cons a fl = singleton a <> fl

a = toFList [1,2,3]
b = cons 3 a

b

[3,1,2,3]

In [31]:
snoc :: FList a -> a -> FList a
snoc fl a = fl <> singleton a

a = toFList [1,2,3]
b = snoc a 3

b

[1,2,3,3]

In [32]:
append :: FList a -> FList a -> FList a
append = (<>)

toFList [1,2,3] `append` toFList [4,5,6]

[1,2,3,4,5,6]

In [35]:
concat :: [FList a] -> FList a
concat [] = emptyList
concat (x:xs) =  x <> concat xs
-- concat xs = foldr (<>) emptyList xs

a = toFList [1,2,3]
b = toFList [4,5,6]
c = [a,b]

concat c

Line 2: Use foldr
Found:
concat [] = emptyList
concat (x : xs) = x <> concat xs
Why not:
concat xs = foldr (<>) emptyList xs

[1,2,3,4,5,6]

In [36]:
head :: FList a -> a
head fl =  fromFList fl !! 0

head $ toFList [1,2,3]

-- take 1 [1,2,3]

1

In [ ]:
tail :: FList a -> FList a
tail fs = toFList (case fromFList fs of
                     [] -> []
                     (_:xs) -> xs)

In [ ]:
foldr :: (a -> b -> b) -> b -> FList a -> b
foldr f b fl = foldr' f b (fromFList fl) where
    foldr' :: (a -> b -> b) -> b -> [a] -> b
    foldr' _ acc [] = acc
    foldr' f acc (x:xs) = f x $ foldr' f acc xs

In [ ]:
reverse :: [a] -> [a]
reverse xs = fromFList $ rev xs where
    rev :: [a] -> FList a
    rev [] = emptyList
    rev (x:xs) = rev xs <> singleton x

reverse [1,2,3]

[3,2,1]

In [38]:
reverseF :: FList a -> FList a
reverseF emptyList = emptyList
reverseF fl = cons (head fl) (reverseF (tail fl))

fromFList $ reverseF $ toFList [1,2,3]

: 